In [ ]:
import os
import yaml

import cv2
import foolbox as fb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf

from adversarial_generators.fgsm import generate_adversarial_images
from model.VGG19 import VGG19
from preprocess.preprocess import load_data

# Config

In [ ]:
with open('config/config.yml', 'r') as stream:
    config = yaml.safe_load(stream)

ROOT_DIRECTORY = os.path.dirname(os.path.abspath('__file__'))

# Load weights to model

In [ ]:
input_shape = (config["img_height"], config["img_width"], 3)
model = VGG19(input_shape = input_shape, num_classes = config["num_classes"])
model.load_weights(config["path_to_weights"])

# Compile Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = config["learning_rate"], decay = config["learning_rate"] / (config["epochs"]))
loss = config["loss_function"]
metrics = config["metrics"]
model.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

# Load data for adversarial training

In [ ]:
path_to_set = os.path.join(ROOT_DIRECTORY, config["path_to_data"])
path_to_train_csv = os.path.join(ROOT_DIRECTORY, config["path_to_train_csv"])
path_to_test_csv = os.path.join(ROOT_DIRECTORY, config["path_to_test_csv"])
(X_train, y_train) = load_data(path_to_train_csv, path_to_set, config["img_width"], config["img_height"])
(X_test, y_test) = load_data(path_to_test_csv, path_to_set, config["img_width"], config["img_height"])
X_test, X_adversarial_train, y_test, y_adversarial_train = train_test_split(X_test, y_test, test_size = 0.5, random_state = 0)

# Normalize the data

In [ ]:
X_train = X_train.astype("float32") / 255.0
X_adversarial_train = X_adversarial_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# One-Hot Encode Target value

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, config["num_classes"])
y_test = tf.keras.utils.to_categorical(y_test, config["num_classes"])

# Plot some adversarial images

In [ ]:
'''
epsilons = [0.001, 0.01, 0.03, 0.1]
index = 213 # Change if you want to see other images 
images = X_test[index:index + 64]
labels = y_test[index:index + 64]
f, axarr = plt.subplots(5, 5, figsize = (30, 20))
for i in range(5):
  axarr[i, 0].imshow(images[i])
  axarr[i, 0].set_xlabel("Original class: {}".format(np.argmax(labels, axis = 1)[i]))
for i, eps in enumerate(epsilons):
  adversarial_images = generate_adversarial_images(images, labels, eps, model).numpy()
  new_predictions = model.predict_on_batch(adversarial_images)
  new_predictions = np.argmax(new_predictions, axis = 1)
  for ax in range(5):
    axarr[ax, i + 1].imshow(adversarial_images[ax])
    axarr[ax, i + 1].set_xlabel("New class: {}".format(new_predictions[ax]))
  f.axes[i + 1].set_title('Eps: {}'.format(eps))
plt.show()
'''

# Duplicate model to Model A and Model B

In [ ]:
# Model A
model_a = tf.keras.models.clone_model(model)
model_a.load_weights(config["path_to_weights"])
model_a.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

# Model B
model_b = tf.keras.models.clone_model(model)
model_b.load_weights(config["path_to_weights"])
model_b.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

# *Adversarial Training*

In [ ]:
X_adversarial_train = tf.convert_to_tensor(X_adversarial_train)
y_adversarial_train = tf.convert_to_tensor(y_adversarial_train)
attack = fb.attacks.LinfPGD(abs_stepsize = 0.0078, steps = 7)
bounds = (0, 1)
preprocessing = dict()
fmodel = fb.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)

## Generate Adversarial Images

In [ ]:
eps = 0.031
_, adversarial_images, indexes_of_wrong_images = attack(fmodel, X_adversarial_train, y_adversarial_train, epsilons = eps)

# Get the images wrongly classified by model A
indexes_of_wrong_images = indexes_of_wrong_images.numpy()
adversarial_images = adversarial_images.numpy()
wrong_classified_images = adversarial_images[indexes_of_wrong_images]

# Convert list of prob to one hot encoding for traing model B
new_predictions = model.predict_on_batch(wrong_classified_images)
new_predictions = tf.keras.utils.to_categorical(np.argmax(new_predictions, axis = 1))
print(len(new_predictions))

## Create Callback for Early Stopping

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)

## Shuffle train data for adversarial training

In [ ]:
# Alpha ratio between train and adv train data
from sklearn.utils import resample

alpha = 3
X_train_new, y_train_new = resample(X_train, y_train, n_samples = alpha * len(wrong_classified_images), random_state = 0)
y_adversarial_train = tf.keras.utils.to_categorical(y_adversarial_train, config["num_classes"])
X_for_B, y_for_B = sklearn.utils.shuffle(
  np.concatenate((X_train_new, wrong_classified_images)),
  np.concatenate((y_train_new, y_adversarial_train[indexes_of_wrong_images])),
  random_state = 0)

## Train Model A

In [ ]:
N = 1
for i in range(N):
  model_a.fit(
    x = wrong_classified_images,
    y = new_predictions,
    batch_size = config["batch_size"],
    verbose = 1,
    validation_split = 0.1,
    callbacks = [callback],
    epochs = config["epochs"])

  model_a.fit(
    x = X_train_new,
    y = y_train_new,
    batch_size = config["batch_size"],
    verbose = 1,
    validation_split = 0.1,
    callbacks = [callback],
    epochs = config["epochs"])


## Train Model B

In [ ]:
model_b.fit(
  x = X_for_B,
  y = y_for_B,
  batch_size = config["batch_size"],
  verbose = 1,
  validation_split = 0.1,
  callbacks = [callback],
  epochs = config["epochs"])

In [ ]:
print('Model A')
model_a.evaluate(X_test, y_test)
print('Model B')
model_b.evaluate(X_test, y_test)

# Save_weights

In [ ]:
path_to_weights_a = os.path.join('..\weights3\\', 'PGD1_alpha={}_N={}_weight_a.h5'.format(alpha, N))
path_to_weights_b = os.path.join('..\weights3\\', 'PGD1_alpha={}_N={}_weight_b.h5'.format(alpha, N))
model_a.save_weights(os.path.join(ROOT_DIRECTORY, path_to_weights_a))
model_b.save_weights(os.path.join(ROOT_DIRECTORY, path_to_weights_b))